In [ ]:
!pip install transformers torch datasets

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load tokenizer and model
model_name = "roberta-base"  # or "roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3, output_hidden_states=True)

In [ ]:
from datasets import load_dataset

# dataset = load_dataset("csv", data_files={"train": "./train/SCOPE_train.csv", "test": "./test/SCOPE_test.csv"})
dataset = load_dataset("csv", data_files={"train": "./augment/SCOPE_aug_train.csv", "test": "./test/SCOPE_test.csv"})

print(dataset)
                                          
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [3]:
from torch.utils.data import DataLoader

train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [4]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [5]:
import torch
from torch.optim import AdamW
from tqdm import tqdm
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = nn.DataParallel(model)
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

num_epochs = 10  # 학습 횟수 설정

for epoch in tqdm(range(num_epochs)):
    model.train()  # 학습 모드 설정
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)  # 실제 정답

        optimizer.zero_grad()  # 기존 gradient 초기화
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)  # 손실 계산
        loss.backward()  # 역전파 수행
        optimizer.step()  # 모델 업데이트

        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

 10%|██████████████████████▍                                                                                                                                                                                                         | 1/10 [02:07<19:03, 127.11s/it]

Epoch 1, Loss: 1.0722


 20%|████████████████████████████████████████████▊                                                                                                                                                                                   | 2/10 [04:03<16:08, 121.05s/it]

Epoch 2, Loss: 0.8851


 30%|███████████████████████████████████████████████████████████████████▏                                                                                                                                                            | 3/10 [06:00<13:53, 119.02s/it]

...

Epoch 8, Loss: 0.0269


 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                      | 9/10 [17:40<01:56, 116.81s/it]

Epoch 9, Loss: 0.0178


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [19:37<00:00, 117.73s/it]

Epoch 10, Loss: 0.0184


In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# F1, Precision, Recall 계산
f1 = f1_score(all_labels, all_preds, average="weighted")
precision = precision_score(all_labels, all_preds, average="weighted")
recall = recall_score(all_labels, all_preds, average="weighted")
accuracy = accuracy_score(all_labels, all_preds)

print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Test F1 Score: 0.3451
Test Precision: 0.3647
Test Recall: 0.3400
Test Accuracy: 0.3400


In [8]:
GPT_zeroshot = [2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0]
f1 = f1_score(all_labels, GPT_zeroshot, average="weighted")
precision = precision_score(all_labels, GPT_zeroshot, average="weighted")
recall = recall_score(all_labels, GPT_zeroshot, average="weighted")

print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

Test F1 Score: 0.3707
Test Precision: 0.6316
Test Recall: 0.4400


In [12]:
GPT_fewshot = [2, 0, 2, 0, 0, 2, 1, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 1, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 1, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0]

f1 = f1_score(all_labels, GPT_fewshot, average="weighted")
precision = precision_score(all_labels, GPT_fewshot, average="weighted")
recall = recall_score(all_labels, GPT_fewshot, average="weighted")
accuracy = accuracy_score(all_labels, GPT_fewshot)


print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Test F1 Score: 0.4257
Test Precision: 0.6300
Test Recall: 0.4700
Test Accuracy: 0.4700
